In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from models import SimpleCNN, MoreComplex
from dataloader import BirdDataset
import os

In [ ]:
# In your Jupyter Notebook
%load_ext autoreload
%autoreload 2

# Set environment variable for more accurate CUDA error messages
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Create the model instance and move it to the device (GPU/CPU)
model = SimpleCNN().to(device)

# Define transformations
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Initialize the dataset and dataloader
train_dataset = BirdDataset(csv_file="train_images.csv", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # or any other appropriate number of epochs
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")